<a href="https://colab.research.google.com/github/sunohk/dataon_pj/blob/main/%ED%95%99%EC%8A%B5_%EC%BD%94%EB%93%9C/6_final_cartogram(%EC%A7%80%EB%8F%84_%EC%8B%9C%EA%B0%81%ED%99%94).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math


In [ ]:
path = '/gdrive/MyDrive/01_공모전/2023_News/Datasets/Cartogram/data/'
nurse_df_raw = pd.read_excel(path+'nurse.xlsx')
stalking_df_raw = pd.read_excel(path+'stalking.xlsx')
rent_df_raw = pd.read_excel(path+'rent.xlsx')
youth_df_raw = pd.read_excel(path+'youth.xlsx')

media_df = pd.read_excel(path + 'media_list.xlsx')
media_df

,user,Address Region,Address SD,Address SGG,category
0,CJB청주방송,충청북도,충청북도,청주,지역
1,JIBS,제주,제주특별자치도,NaN,지역
2,JTBC,전국,NaN,NaN,방송사
3,KBS,전국,NaN,NaN,방송사
4,KBS뉴스,전국,NaN,NaN,방송사
...,...,...,...,...,...
139,한국경제TV,전국,NaN,NaN,경제
140,한국일보,전국,NaN,NaN,전국
141,헤럴드POP,전국,NaN,NaN,"전문지(연예, 스포츠 등)"
142,헤럴드경제,전국,NaN,NaN,경제


In [ ]:
#df와 결합 합수
def merge_df_publisher(df):
  merged_df = pd.merge(df, media_df, left_on='작성자', right_on = 'user', how = 'inner')
  merged_df.drop(columns=['clean_text',	'clean_title',	'clean_text_2',	'clean_contents',	'summary','user',
                        'Address SD',	'Address SGG'],inplace=True)
  merged_df.columns = ['Date', 'Publisher', 'Title', 'Text', 'Region', 'Type' ]
  return merged_df

nurse_df = merge_df_publisher(nurse_df_raw)
stalking_df = merge_df_publisher(stalking_df_raw)
rent_df = merge_df_publisher(rent_df_raw)
youth_df = merge_df_publisher(youth_df_raw)

In [ ]:
youth_df.head()

,Date,Publisher,Title,Text,Region,Type
0,2019/05/01,강원일보,[강릉]청년정책보고회 개최,【강릉】강릉시는 지난 30일 시청 18층 전망대에서 현재 청년 관련 정책을 추진하고...,강원,지역
1,2019/05/16,강원일보,[정선]청년정책위 운영 박차,【정선】정선군이 인구 감소와 초고령화 사회 문제를 극복하고 청년 지원을 위해 추진 ...,강원,지역
2,2019/05/24,강원일보,[강릉]강릉청년정책위원회 출범,【강릉】강릉시는 23일 시청 8층 상황실에서 강릉 청년들과 청년 정책의 분야별 전문...,강원,지역
3,2019/05/27,강원일보,오늘 도의회 청년정책 토론회,도의회는 27일 오후 2시 의회 지하 1층 세미나실에서 `강원도 청년정책 토론회'를...,강원,지역
4,2019/05/28,강원일보,[포토뉴스]강원도청년정책 토론회,"강원도 청년정책 토론회가 27일 도의회에서 한금석 도의장, 허소영·조형연·조성호 도...",강원,지역


In [ ]:
#확인
nurse_df[nurse_df['Region'] == '충청북도'].head()

,Date,Publisher,Title,Text,Region,Type
10367,2023/05/03,CJB청주방송,충북 의료계 `간호법 반대` 도심집회,충북보건복지의료연대 등 2백여명의 충북지역의료계는 오늘(3) 도청 앞에서 집회를 열...,충청북도,지역
10368,2023/05/04,CJB청주방송,"민주 김민석 ""윤 대통령 `간호법` 거부권 행사 이해하기 어렵다""",김민석 더불어민주당 정책위의장은 윤석열 대통령의 간호법 거부권 행사 가능성에 대해 ...,충청북도,지역
10369,2023/05/16,CJB청주방송,"尹 ""간호법, 갈등·국민 불안 초래한다""..민주 ""혼란 더 악화시킨다""",윤석열 대통령이 16일 간호법 제정안에 대해 거부권을 행사했습니다. 윤 대통령은 용...,충청북도,지역
10370,2023/05/17,CJB청주방송,간호협회 “면허증 반납 운동 전개... 채혈 등 불법 지시 거부”,윤석열 대통령이 지난 16일 간호법 제정안에 대해 재의 요구권(거부권)을 행사한 가...,충청북도,지역
10371,2023/05/30,CJB청주방송,간호법 오늘 국회서 재의결..국민의힘 이탈표 나올까?,30일 오후에 열리는 국회 본회의에서 대통령이 거부권을 행사한 간호법에 대한 재의결...,충청북도,지역


In [ ]:
#언론사가 '전국'인 row subset
def subset_all(df):
  subset_df = df[df['Region'] == '전국']
  return subset_df

#언론사가 '전국'이 아닌 row subset
def subset_regions(df):
  subset_df = df[df['Region'] != '전국']
  return subset_df


#subset all df
sub_nurse_df = subset_all(nurse_df) #9484
sub2_nurse_df = subset_regions(nurse_df) #895
sub_stalking_df = subset_all(stalking_df) #12541
sub2_stalking_df = subset_regions(stalking_df) #1446
sub_rent_df = subset_all(rent_df) #21525
sub2_rent_df = subset_regions(rent_df) #2290
sub_youth_df = subset_all(youth_df) #16536
sub2_youth_df = subset_regions(youth_df) #2321

sub_nurse_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9484 entries, 0 to 10378
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Date       9484 non-null   object
 1   Publisher  9484 non-null   object
 2   Title      9484 non-null   object
 3   Text       9484 non-null   object
 4   Region     9484 non-null   object
 5   Type       9484 non-null   object
dtypes: object(6)
memory usage: 518.7+ KB


In [ ]:
sub_nurse_df

,Date,Publisher,Title,Text,Region,Type
0,2015/06/17,국민일보,"선진국 간병 문화, 감염통제 철저",한국의 한 병원에서 가족이 간병하고 있다. 국민일보DB 이번 중동호흡기증후군(메르...,전국,전국
1,2019/10/30,국민일보,보건의료 개혁 위한 간호법 제정 선포식,30일 서울 광화문광장에서 열린 '2019 간호정책선포식'에서 참석자들이 대한민국 ...,전국,전국
2,2019/10/30,국민일보,[포토] “간호법 제정하라” 5만여명 집회,간호사와 간호대학생들이 30일 서울 광화문광장에서 열린 ‘2019 간호정책 선포식’...,전국,전국
3,2021/12/01,국민일보,[포토] “간호법 제정해야” 절박한 목소리,대한간호협회 관계자가 1일 국회 앞에서 열린 ‘간호법 제정과 불법진료·불법의료기관 ...,전국,전국
4,2022/01/17,국민일보,이재명 “샤먼이 국정에 영향 미쳐선 안 돼…간호사법 제정 공감”,이재명 더불어민주당 대선후보가 17일 서울 강서구 이화여대 서울병원에서 청년 간호사...,전국,전국
...,...,...,...,...,...,...
10374,2023/05/16,이코노미스트,"[속보] 尹대통령, 간호법 제정안 거부권 행사…취임 후 2번째",[이코노미스트 이혜리 기자] 윤석열 대통령이 16일 간호법 제정안에 대해 법률안 거...,전국,경제
10375,2023/05/19,주간경향,여냐 야냐 ‘어느 쪽이 더 싫으냐’ 경쟁,총선 1년 남았는데 비호감 최악…여야 사령탑 리더십 부재 총선이 1년여도 남지 않았...,전국,전국
10376,2023/06/07,주간경향,쏠리면 위험하다[편집실에서],"의사의 인기가 하늘을 찌릅니다. ‘N수’는 기본이고, 서울 명문대 재학생들이 학교를...",전국,전국
10377,2023/05/21,MBC연예,`스트레이트` 윤석열 정부 1년...`정치`가 사라졌다,오늘(21일) 방송되는 MBC 탐사기획 ‘스트레이트’에서는 혐오와 극한의 대결 속...,전국,방송사


In [ ]:
sub2_nurse_df

,Date,Publisher,Title,Text,Region,Type
1734,2015/08/07,부산일보,[볼만한TV프로그램] 오! 마이 베이비(SBS 8일 오후 5시) 外,오락 오! 마이 베이비(SBS 8일 오후 5시) 일란성 쌍둥이 라희와 라율의 달라도...,부산,지역
1735,2017/06/22,부산일보,[이 주의 새 책] 올빼미는 밤에만 사냥한다 外,■올빼미는 밤에만 사냥한다 겨울을 앞둔 노르웨이 숲에서 알몸 상태로 목이 졸린 채 ...,부산,지역
1736,2018/11/06,부산일보,가야대 간호학과 권영채 교수 및 재학생 2018년 간호정책 선포식 참석,대한간호협회(회장 신경림)는 1일 서울 장충체육관에서 정치계와 보건의료 관련 단체장...,부산,지역
1737,2019/11/06,부산일보,대한간호협회 신경림 회장 대동대학교 간호학부 특강,"대동대학교(총장 배순철) 간호학부에서는 지난 5일, 대한간호협회 신경림 회장님을 강...",부산,지역
1738,2021/05/13,부산일보,[기고] “nursing korea! 간호사가 대한민국을 간호하겠습니다”,황지원 부산광역시간호사회 회장 올해 대한간호협회에서 내건 슬로건이다. 간호사는 항상...,부산,지역
...,...,...,...,...,...,...
10368,2023/05/04,CJB청주방송,"민주 김민석 ""윤 대통령 `간호법` 거부권 행사 이해하기 어렵다""",김민석 더불어민주당 정책위의장은 윤석열 대통령의 간호법 거부권 행사 가능성에 대해 ...,충청북도,지역
10369,2023/05/16,CJB청주방송,"尹 ""간호법, 갈등·국민 불안 초래한다""..민주 ""혼란 더 악화시킨다""",윤석열 대통령이 16일 간호법 제정안에 대해 거부권을 행사했습니다. 윤 대통령은 용...,충청북도,지역
10370,2023/05/17,CJB청주방송,간호협회 “면허증 반납 운동 전개... 채혈 등 불법 지시 거부”,윤석열 대통령이 지난 16일 간호법 제정안에 대해 재의 요구권(거부권)을 행사한 가...,충청북도,지역
10371,2023/05/30,CJB청주방송,간호법 오늘 국회서 재의결..국민의힘 이탈표 나올까?,30일 오후에 열리는 국회 본회의에서 대통령이 거부권을 행사한 간호법에 대한 재의결...,충청북도,지역


# 2. 제목, 내용 컬럼 활용 지역 컬럼 생성하기
- 지역 정보 출처 : 행안부 참고(https://www.laiis.go.kr/lips/mlo/wco/wholeCountryList.do)

 - 특별시 : 서울특별시
 - 광역시 : 부산광역시, 인천광역시, 대구광역시, 대전광역시, 광주광역시, 울산광역시
 - 특별자치시 : 세종특별자치시
 - 도 : 경기도, 충청북도, 충청남도, 전라북도, 전라남도, 경상북도, 경상남도
 - 특별자치도 : 강원특별자치도, 제주특별자치도

In [ ]:
# 주어진 지역 리스트
region_list = ['서울', '부산', '인천', '대구', '대전', '광주', '울산', '세종', '경기', '충청', '전라', '경상', '강원', '제주']
# region_list = ['서울', '부산', '인천', '대구', '대전', '광주', '울산', '세종', '경기', '충청남도', '충청북도' '전라남도', '전라북도' '경상남도', '경상북도' '강원', '제주']


## 제목에 가중치 2배 부여한 ver.

In [ ]:
%%capture

# max_region_title 초기화
max_region_title = 0

def find_max_region(df):
    title_text = df['Title']  # clean_title 대체
    contents_text = df['Text']  # clean_text_2 대체
    # '제목' 열에서 각 지역 이름의 출현 횟수에 가중치 2를 곱한 값을 저장한 딕셔너리.
    region_counts_title_weighted = {region: title_text.count(region) * 2 for region in region_list}  # 가중치 2 부여

    #'내용' 열에서 각 지역 이름의 출현 횟수를 저장한 딕셔너리.
    region_counts_text = {region: contents_text.count(region) for region in region_list}  # 딕셔너리 형식

    # '제목' 열에서 가중치를 적용한 가장 많이 출현한 지역 이름의 출현 횟수 중 가장 큰 값을 저장
    max_region_title_weighted = max(region_counts_title_weighted.values(), default=0)

    # 내용' 열에서 가장 많이 출현한 지역 이름의 출현 횟수 중 가장 큰 값을 저장
    max_region_text = max(region_counts_text.values(), default=0)

    if max(max_region_title_weighted, max_region_text) < 2:
        return '전국'
    elif max_region_title_weighted >= 2 and max_region_text < 2:
        return max(region_counts_text, key=region_counts_text.get)  # 제목 기반 최대인 키를 반환
    elif max_region_text >= 2 and max_region_title_weighted < 2:
        return max(region_counts_title_weighted, key=region_counts_title_weighted.get)  # 내용 기반 값이 최대인 키를 반환
    else:
        return max(max(region_counts_title_weighted, key=region_counts_title_weighted.get),
                   max(region_counts_text, key=region_counts_text.get))

# for each subset df 'topic_region' 컬럼 생성
sub_nurse_df['topic_region'] = sub_nurse_df.apply(find_max_region, axis=1)
sub_stalking_df['topic_region'] = sub_stalking_df.apply(find_max_region, axis=1)
sub_rent_df['topic_region'] = sub_rent_df.apply(find_max_region, axis=1)
sub_youth_df['topic_region'] = sub_youth_df.apply(find_max_region, axis=1)


In [ ]:
sub_nurse_df.head()

,Date,Publisher,Title,Text,Type,Region,topic_region
0,2015/06/17,국민일보,"선진국 간병 문화, 감염통제 철저",한국의 한 병원에서 가족이 간병하고 있다. 국민일보DB 이번 중동호흡기증후군(메르...,전국,전국,전국
1,2019/10/30,국민일보,보건의료 개혁 위한 간호법 제정 선포식,30일 서울 광화문광장에서 열린 '2019 간호정책선포식'에서 참석자들이 대한민국 ...,전국,전국,전국
2,2019/10/30,국민일보,[포토] “간호법 제정하라” 5만여명 집회,간호사와 간호대학생들이 30일 서울 광화문광장에서 열린 ‘2019 간호정책 선포식’...,전국,전국,전국
3,2021/12/01,국민일보,[포토] “간호법 제정해야” 절박한 목소리,대한간호협회 관계자가 1일 국회 앞에서 열린 ‘간호법 제정과 불법진료·불법의료기관 ...,전국,전국,전국
4,2022/01/17,국민일보,이재명 “샤먼이 국정에 영향 미쳐선 안 돼…간호사법 제정 공감”,이재명 더불어민주당 대선후보가 17일 서울 강서구 이화여대 서울병원에서 청년 간호사...,전국,서울,서울


In [ ]:
sub_nurse_df['topic_region'].unique()

array(['전국', '서울', '대구', '제주', '충청', '부산', '강원', '인천', '경기', '대전', '광주',
       '세종', '울산', '경상'], dtype=object)

In [ ]:
#전국에서 가중치 기준으로 재분류된 뉴스는 2684
sub_nurse_df[sub_nurse_df['topic_region'] != '전국'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2734 entries, 4 to 10376
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Date          2734 non-null   object
 1   Publisher     2734 non-null   object
 2   Title         2734 non-null   object
 3   Text          2734 non-null   object
 4   Type          2734 non-null   object
 5   Region        2734 non-null   object
 6   topic_region  2734 non-null   object
dtypes: object(7)
memory usage: 170.9+ KB


## 빈도수 계산

In [ ]:
# List of regions to split
regions_to_split = ['충청', '경상', '전라']

#충청, 경상, 전라 남북 나누기
def split_region(df, region_name):
    if region_name in df['지역'].values:
        index = df[df['지역'] == region_name].index[0]
        original_count = df.loc[index, '빈도수']
        half_count = math.ceil(original_count / 2)
        df.loc[index:index, '빈도수'] = half_count
        df.loc[index:index, '지역'] = region_name + '북도'
        new_row = {'지역': region_name + '남도', '빈도수': half_count}
        df = df.append(new_row, ignore_index=True)
    return df

def region_counts(df, df2):
  region_counts = df['topic_region'].value_counts().reset_index()
  region_counts.columns = ['지역', '빈도수']
  # region_counts = region_counts.sort_values(by='빈도수', ascending=False)

  for region in regions_to_split:
      region_counts = split_region(region_counts, region)

  # 신문사 기반으로 분류된 subset df2 count
  region_counts2 = df2['Region'].value_counts().reset_index()
  region_counts2.columns = ['지역', '빈도수']
  # region_counts2 = region_counts2.sort_values(by='빈도수', ascending=False)

  #merge신문사 기반으로 분류된 subset_df2와 결합 합수
  merged_df = pd.merge(region_counts, region_counts2, on='지역', suffixes=('_df1', '_df2'), how='outer')

  # Fill NaN values with 0 and add the counts
  merged_df['빈도수_df1'].fillna(0, inplace=True)
  merged_df['빈도수_df2'].fillna(0, inplace=True)
  merged_df['빈도수'] = merged_df['빈도수_df1'] + merged_df['빈도수_df2']

  # Drop the intermediate columns
  merged_df.drop(['빈도수_df1', '빈도수_df2'], axis=1, inplace=True)

    #전국 미포함
  merged_df = merged_df.iloc[1:].sort_values(by='빈도수', ascending=False)
  return merged_df

  # new_subset_df = pd.concat([region_counts, region_counts2])
  # return new_subset_df

nurse_region_counts = region_counts(sub_nurse_df, sub2_nurse_df)
stalking_region_counts = region_counts(sub_stalking_df, sub2_stalking_df)
rent_region_counts = region_counts(sub_rent_df, sub2_rent_df)
youth_region_counts = region_counts(sub_youth_df, sub2_youth_df )

<ipython-input-173-6f74fd07e582>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
<ipython-input-173-6f74fd07e582>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
<ipython-input-173-6f74fd07e582>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
<ipython-input-173-6f74fd07e582>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
<ipython-input-173-6f74fd07e582>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future versio

In [ ]:
nurse_region_counts

,지역,빈도수
1,서울,2931.0
3,대구,171.0
2,부산,157.0
4,강원,130.0
6,대전,69.0
5,광주,62.0
11,인천,43.0
10,제주,14.0
7,세종,12.0
8,울산,12.0


In [ ]:
#전국 제외 데이터 엑셀로 저장
nurse_region_counts.to_excel('/gdrive/MyDrive/01_공모전/2023_News/Datasets/Cartogram/nurse_cartogram.xlsx', index=False)
stalking_region_counts.to_excel('/gdrive/MyDrive/01_공모전/2023_News/Datasets/Cartogram/stalking_cartogram.xlsx', index=False)
rent_region_counts.to_excel('/gdrive/MyDrive/01_공모전/2023_News/Datasets/Cartogram/rent_cartogram.xlsx', index=False)
youth_region_counts.to_excel('/gdrive/MyDrive/01_공모전/2023_News/Datasets/Cartogram/youth_cartogram.xlsx', index=False)


#하나의 DF

In [ ]:
nurse_region_counts['Topic'] = '간호법'
stalking_region_counts['Topic'] = '스토킹처벌'
rent_region_counts['Topic'] = '전세사기'
youth_region_counts['Topic'] = '청년정책'

# Concatenate the DataFrames vertically
result_region_counts = pd.concat([nurse_region_counts, stalking_region_counts, rent_region_counts, youth_region_counts]).reset_index(drop=True)
result_region_counts

,지역,빈도수,Topic
0,서울,2931.0,간호법
1,대구,171.0,간호법
2,부산,157.0,간호법
3,강원,130.0,간호법
4,대전,69.0,간호법
...,...,...,...
60,충청북도,21.0,청년정책
61,충청남도,15.0,청년정책
62,전라남도,12.0,청년정책
63,경상북도,9.0,청년정책


In [ ]:
result_region_counts.to_excel('/gdrive/MyDrive/01_공모전/2023_News/Datasets/Cartogram/all_topics_cartogram.xlsx', index=False)